# HyperParameters Tuning

## Initial Operations

### Import the libraries

In [ ]:
!pip install keras_tuner
import os
import numpy as np
import random
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tfk = tf.keras
tfkl = tf.keras.layers

### Set random seed for reproducibility

In [ ]:
# Random seed for reproducibility
seed = 69

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Metadata

In [ ]:
training_dir = './GitHub/LeafClassification-AN2DL/Data/dataset_no_corrupted/training'
validation_dir = './GitHub/LeafClassification-AN2DL/Data/dataset_no_corrupted/validation'

In [ ]:
input_shape = (256, 256, 3)
epochs = 200

## Model - Transfer Learning from InceptionResNetV2

### Data Generators

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

train_data_gen = ImageDataGenerator(rotation_range=90,
                                    height_shift_range=100,
                                    width_shift_range=100,
                                    zoom_range=0.5,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    shear_range = 0.25,
                                    fill_mode='reflect',
                                    brightness_range=[0.5,1.5],
                                    preprocessing_function = preprocess_input)
val_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None,
                                               class_mode='categorical',
                                               batch_size=128,
                                               shuffle=True,
                                               seed=seed)
validation_gen = val_data_gen.flow_from_directory(directory=validation_dir,
                                           target_size=(256,256),
                                           color_mode='rgb',
                                           classes=None,
                                           class_mode='categorical',
                                           batch_size=128,
                                           shuffle=True,
                                           seed=seed)

### Class Weight (for Unbalanced Classes)

In [ ]:
classes = dict()
for label in sorted(os.listdir(training_dir)):
  classes[label] = len(os.listdir(training_dir + "/" + label))
total = 0
class_weight = dict()
for i, samples_number in enumerate(classes.values()):
    class_weight[i] = 1/samples_number
    total += samples_number
class_weight = {key:value*total/14 for key, value in class_weight.items()}

### Supernet (InceptionResNetV2)

In [ ]:
# Download the supernet
supernet = tfk.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3)
)

87924736/87910968 [==============================] - 1s 0us/step


### Model Definition

In [ ]:
# Build the model
def build_model(hp):
  # Fine Tuning - Freeze the first layers
  fine_tuning_parameters = hp.Choice('fine_tuning_parameters',values = [460,510,600])
  supernet.trainable = True
  for i, layer in enumerate(supernet.layers[:fine_tuning_parameters]):
    layer.trainable=False
  
  input_layer = tfk.Input(shape=input_shape)
  features_extractor = supernet(input_layer)
  flattening = tfkl.Flatten(name='Flattening')(features_extractor)
  dp_parameter1 = hp.Float("dp_parameter1",min_value = 0, max_value = 1)
  dropuout_flattening = tfkl.Dropout(dp_parameter1, seed=seed)(flattening)
  units_parameter1 = hp.Int('units_parameter1', min_value=32, max_value=512, step=32)
  dense1 = tfkl.Dense(
      units_parameter1, 
      activation='relu',
      kernel_initializer = tfk.initializers.GlorotUniform(seed))(dropuout_flattening)
  dp_parameter2 = hp.Float("dp_parameter2",min_value = 0, max_value = 1)
  dropout_dense1 = tfkl.Dropout(dp_parameter2, seed=seed)(dense1)
  units_parameter2 = hp.Int('units_parameter2', min_value=32, max_value=512, step=32)
  dense2 = tfkl.Dense(
      units_parameter2, 
      activation='relu',
      kernel_initializer = tfk.initializers.GlorotUniform(seed))(dropout_dense1)
  dp_parameter3 = hp.Float("dp_parameter3",min_value = 0, max_value = 1)
  dropout_dense2 = tfkl.Dropout(dp_parameter3, seed=seed)(dense2)
  units_parameter3 = hp.Int('units_parameter3', min_value=32, max_value=512, step=32)
  dense3 = tfkl.Dense(
      units_parameter3, 
      activation='relu',
      kernel_initializer = tfk.initializers.GlorotUniform(seed))(dropout_dense2)
  dp_parameter4 = hp.Float("dp_parameter4",min_value = 0, max_value = 1)
  dropout_dense3 = tfkl.Dropout(dp_parameter4, seed=seed)(dense3)
  output_layer = tfkl.Dense(
      14, 
      activation='softmax',
      kernel_initializer = tfk.initializers.GlorotUniform(seed))(dropout_dense3)
  model = tfk.Model(inputs=input_layer, outputs=output_layer, name='InceptionResNetV2')
  model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

  return model

#### Tuning of the Parameters

In [ ]:
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=30,
                     project_name='InceptionResNetV2')
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5)
tuner.search(x = train_gen, epochs=50, validation_data = validation_gen, callbacks=[early_stopping])

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps)